- 그룹 연산: 데이터 집계 변환 등의 작업을 한 번에 처리할 수 있는 기능
- 분할 + 반영 + 결합(Split + Apply + Combine)
  * 분할: 어떤 기준으로 데이터를 나누는 것.
  * 반영: 함수를 적용하여 데이터를 처리하는 것.
  * 결합: 처리한 결과를 합치는 것.
- 그룹 연산은 SQL과 유사한 기능.

# 11-1. 데이터 집계

 - 집계: 수집한 데이터를 바탕으로 평균이나 합 등을 구하여 의미있는 값을 도출해내는 것.
 - 집계 기능의 장점: 전체 데이터를 요약, 정리할 수 있어 데이터 분석을 편리하게 해 줌.

In [ ]:
import pandas as pd
df = pd.read_csv("./data/gapminder.tsv", sep = '\t')
print(df.head())

In [ ]:
#year 열을 기준으로 데이터를 그룹화한 다음, lifeExp 열의 평균 구하기.
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

#### 분할 - 반영 - 결합 과정 살펴보기: groupby 메서드

 - avg_life_exp_by_year를 구하는 과정을 groupby와 유사한 메서드를 순서대로 실행하며 알아보자.

In [ ]:
#1. 분할
years = df.year.unique()  #year 컬럼 데이터를 중복 없이 추출.
print(years)

In [ ]:
#2. 반영
y1952 = df.loc[df.year == 1952, :]  #1952년 데이터 추출
print(y1952.head())

In [ ]:
#1952년 데이터에서 lifeExp열의 평균값 구하기.
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

In [ ]:
#나머지 연도들도 이와 같이 구하기를 완료하면 '반영' 작업 끝.
y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :]
y1962_mean = y1962.lifeExp.mean()
print(y1962_mean)

y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()
print(y2007_mean)

In [ ]:
#3. 결합: 연도별로 계산한 lifeExp의 평균값을 한 데이터프레임으로 합침.
df2 = pd.DataFrame({"year": [1952, 1957, 1962, 2007],
                    "": [y1952_mean, y1957_mean, y1962_mean, y2007_mean]})
print(df2)

- groupby 메서드와 함께 사용하는 집계 메서드.

    - count: 결측치를 제외한 데이터 수 반환.
    - size: 결측치를 포함한 데이터 수 반환.
    - mean: 평균값 반환.
    - std: 표준편차 반환.
    - min: 최솟값 반환.
    - quantile(q = 0.25): 백분위수 25%
    - quantile(q = 0.5): 백분위수 50%
    - quantile(q = 0.75): 백분위수 75%
    - max: 최댓값 반환.
    - sum: 전체 합 반환.
    - var: 분산 반환.
    - sem: 평균의 표준편차 반환.
    - describe: 데이터 수, 평균, 표준편차, 최솟값, 백분위수(25%, 50%, 75%), 최댓값 모두 반환,
    - first: 첫 번째 행 반환.
    - last: 마지막 행 반환.
    - nth: n번째 행 반환.

- agg 메서드로 사용자 함수와 groupby 메서드 조합하기.

In [ ]:
#평균값을 구하는 사용자 함수와 groupby 메서드.
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
        
    return sum / n

In [ ]:
#groupby 메서드와 my_mean 함수를 조합하기 위해 agg 메서드 사용 ==> mean 메서드와 결과 동일.
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

 - 2개의 인자값을 받아 처리하는 사용자 함수와 groupby 메서드.

In [ ]:
#첫 번째 인자로 받은 열의 평균값을 구하여, 두 번째 인자로 받은 값과의 차이를 계산하여 반환하는 함수.
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [ ]:
#연도별 평균 수명에서 전체 평균 수명을 뺀 값을 구한 것.
global_mean = df.lifeExp.mean()
print(global_mean)

                                            #agg()에 (함수, 전체 평균 수명값) 전달.
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean) 
print(agg_mean_diff)

- 여러 개의 집계 메서드 한 번에 사용하기: 집계 메서드를 리스트 또는 딕셔너리에 담아 agg 메서드에 전달.

In [ ]:
#연도별로 그룹화한 lifeExp 열의 0이 아닌 값의 개수, 평균, 표준편차를 한 번에 계산하여 출력.

#numpy 메서드들을 리스트에 담아 agg 메서드에 전달.
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

In [ ]:
#집계 메서드를 딕셔너리에 담아 agg 메서드에 전달.

#key: 집계 메서드를 적용할 열 이름, value: 집계 메서드.
gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'})
print(gdf_dict)

# 11-2. 데이터 변환

 - 데이터 변환 메서드는 데이터와 메서드를 1:1로 대응시켜 계산함.
 - 따라서 데이터 양이 줄어들지 않음.

 - 표준점수 계산하기.

In [ ]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [ ]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

In [ ]:
#원본 데이터프레임(df)의 데이터 크기 VS 변환한 데이터프레임(transform_z)의 데이터 크기 비교.
print(df.shape)

print(transform_z.shape)  #행 개수가 줄어들지 않았음.

 - 결측치를 평균값으로 처리하기.

In [ ]:
#tips 데이터에서 10개의 행 데이터만 가져온 다음, total_bill 열의 값 4개를 임의로 선택하여 결측치로 바꾸기.
import seaborn as sns
import numpy as  np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
print(tips_10)

    <tip> 
    permutation: 주어진 데이터를 섞은 후 출력하는 메서드.
                 어차피 shuffle 의 결과는 순열중에 하나이기 때문에 결과 자체는 같다.

    shuffle과의 차이점: permutation은 array를 복사하여 리턴한다(즉, 원본은 유지가 됨). 
                       반면에 shffle은 원본을 바꿈.

In [ ]:
#단, 데이터를 단순히 평균값으로 채우면 안 됨.
#tips_10 데이터는 여성보다 남성이 더 많음. 
#즉, 남녀를 구분하여 total_bill 열의 평균값을 구하지 않으면, 여성 데이터가 남성 데이터의 영향을 많이 받아 여성의 데이터가 훼손될 수 있음.

#성별로 그룹화한 다음 각 열의 데이터 수를 구하기.
count_sex = tips_10.groupby('sex').count()
print(count_sex)

In [ ]:
#성별을 구분하여 total_bill 열의 데이터를 받아 평균값을 구하는 함수.
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [ ]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean

print(tips_10)